In [ ]:
import torch
#add modular_dnn to path
import sys
sys.path.append('/mnt/ssd-apollo/jake/rib/experiments/counterexamples')
from modular_dnn import BlockDiagonalDNN, RandomVectorDataset
from matplotlib import pyplot as plt
from rib.hook_manager import HookedModel, Hook
from torch.utils.data import DataLoader, Dataset
from rib.hook_fns import acts_pre_forward_hook_fn


In [ ]:
from .modular_dnn import BlockDiagonalDNN, RandomVectorDataset

In [ ]:
results1 = torch.load('/mnt/ssd-apollo/jake/rib/experiments/counterexamples/results/small_modular_dnn_seed100_n20_k10_layers4_random_act_relu_dsize_1000_bias0/rib_graph.pt')
results2 = torch.load('/mnt/ssd-apollo/jake/rib/experiments/counterexamples/results/small_modular_dnn_seed100_n20_k10_layers4_random_act_relu_dsize_1000_bias0_final_layer_fixed/rib_graph.pt')
results1.keys()
plt.loglog(results1['rib_edges']['squared'][2][1].flatten(),results2['rib_edges']['squared'][2][1].flatten(), '.')
plt.title('ratio of edges with and without final layer rotated, newest commit, squared mode')
# plt.plot([results1['rib_edges']['squared'][2][1].min(), results1['rib_edges']['squared'][2][1].max()],[results1['rib_edges']['squared'][2][1].min(), results1['rib_edges']['squared'][2][1].max()],'k--')

In [ ]:
results.keys()

In [ ]:
plt.imshow(results['rib_edges'][2][1])
plt.colorbar()

In [ ]:
results['model_config_dict']

In [ ]:
plt.imshow(results['gram_matrices']['output'].abs().log())
plt.colorbar()

In [ ]:
plt.imshow(results['gram_matrices']['layers.0'].abs().log())
plt.colorbar()

In [ ]:
U,S,V = torch.svd(results['gram_matrices']['layers.0'])
plt.imshow(U)
plt.show()
plt.imshow(V/U)
plt.colorbar()

In [ ]:
n = results['model_config_dict']['n']
k = results['model_config_dict']['k']
layers = results['model_config_dict']['layers']
mlp = BlockDiagonalDNN(layers = layers, n = n, k = k)
hooked_mlp = HookedModel(mlp)
mlp.load_state_dict(results['mlp'])
dataset = RandomVectorDataset(n, k, results['model_config_dict']['dataset_size'],results['model_config_dict']['data_variances'], dtype=results['model_config_dict']['dtype'])

dataloader = DataLoader(dataset, batch_size=results['model_config_dict']['dataset_size'], shuffle=True)

In [ ]:
class OnesDataset(Dataset):
    def __init__(self, n):
        self.data = torch.ones(n)
        self.label = torch.tensor(1)

    def __len__(self):
        return 1  # Since there's only one datapoint

    def __getitem__(self, idx):
        return self.data, self.label

class IdentityDataset(Dataset):
    def __init__(self, n):
        self.data = torch.eye(n)
        self.label = torch.tensor(1)

    def __len__(self):
        return 1  # Since there's only one datapoint

    def __getitem__(self, idx):
        return self.data, self.label



# Create the dataset and dataloader
# dataset = OnesDataset(n)
# dataloader = DataLoader(dataset, batch_size=1)

In [ ]:
n

In [ ]:
dataset = IdentityDataset(n)
dataloader = DataLoader(dataset, batch_size=n)
for data,_ in dataloader:
    print(data) 
    print(mlp(data))
    print(hooked_mlp(data))
    break

In [ ]:
layers_to_hook = [f'layers.{i}' for i in range(4)]
activation_hooks = []
for layer in layers_to_hook:
    activation_hooks.append(
        Hook(
            name=layer, 
            data_key="activations", 
            fn=acts_pre_forward_hook_fn, 
            module_name=layer,
            fn_kwargs={}
        )
    )
for inputs, _ in dataloader:
    _ = hooked_mlp(inputs, hooks=activation_hooks)


In [ ]:
activations = {}
for layer in layers_to_hook:
    activations[layer] = hooked_mlp.hooked_data[layer]['activations'][0]


In [ ]:
activations['layers.0'].std(dim=0)

In [ ]:
plt.imshow(results['eigenvectors'][0]['U'].abs().log())
plt.colorbar()

In [ ]:
plt.imshow(results['pca_edges'][0][1].abs().log())
plt.colorbar()

In [ ]:
plt.imshow(results['interaction_rotations'][1]['C'].abs().log())
plt.colorbar()

In [ ]:
for i in range(4):
    U,S,Vt = torch.svd(activations[f'layers.{i}'])# - activations[f'layers.{i}'].mean(dim=0))
    print(S)
    plt.scatter(range(len(S)),S.log())
    plt.show()
    plt.imshow(Vt.abs().log())
    plt.colorbar()
    plt.show()